In [3]:
import os, sys
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
from config import SciFactT5Config
import definitions

## Set Project directory

In [4]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from config import SciFactT5Config
import definitions
sys.path.append(os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR))
print(definitions.PROJECT_VARS.ROOT_DIR)

NameError: name 'definitions' is not defined

## Load dataset for Scifact

In [ ]:
from verisci.covid import AbstractRetriever, RationaleSelector, LabelPredictor
from verisci.evaluate.lib.data import GoldDataset

In [ ]:
def get_claim_label_from_jsonl(dataset_jsonl):
    """_summary_

    Args:
        dataset_jsonl (_type_): _description_

    Returns:
        _type_: _description_
    """
    claim_label_list_train = []
    for cur_claim in dataset_jsonl:
        claim_txt = cur_claim.claim

        for doc_id, evidence in cur_claim.evidence.items():

            ev_doc = cur_claim.release.corpus.get_document(doc_id)

            claim_label = evidence.label.name

            tmp_dic = {"claim" : claim_txt, "label" : claim_label}

            claim_label_list_train.append(tmp_dic)
    return claim_label_list_train

In [ ]:
def get_claim_label_evidence_from_jsonl(dataset_jsonl, source):
    """_summary_

    Args:
        dataset_jsonl (_type_): _description_
        source (_type_): _description_

    Returns:
        _type_: _description_
    """
    claim_label_list_train = []
    for cur_claim in dataset_jsonl:
        claim_txt = cur_claim.claim
        for doc_id, evidence in cur_claim.evidence.items():
            ev_doc = claim_train.release.corpus.get_document(doc_id)
            claim_label = evidence.label.name
            list_rationales = []
            for i, sents in enumerate(evidence.rationales):
                list_rationales = [sent for i, sent in enumerate(ev_doc.sentences) if i in sents]
            tmp_dic = {"claim" : claim_txt, "label" : claim_label, "list_rationales" :list_rationales, "source" :source}
            claim_label_list_train.append(tmp_dic)
    return claim_label_list_train

In [ ]:

cfg= SciFactT5Config()
ds_train = GoldDataset(cfg.target_dataset.loc_target_dataset_corpus,
                    cfg.target_dataset.loc_target_dataset_train)
claim_train = ds_train.get_claim(39)

dic_train = get_claim_label_evidence_from_jsonl(ds_train, source = "train")

SciFactT5Config(target_dataset=ScifactTargetDataset(base_dir='/home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo/target_system/scifact/data', loc_target_dataset_corpus='/home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo/target_system/scifact/data/corpus.jsonl', loc_target_dataset_train='/home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo/target_system/scifact/data/claims_train.jsonl', loc_target_dataset_dev='/home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo/target_system/scifact/data/claims_dev.jsonl'), target_model=ScifactTargetModel(base_dir='/home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo/target_system/model', loc_label_model='/home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo/target_system/model/label_roberta_large_fever_scifact', loc_rationale_model='/home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo/target_system/model/rationale_roberta_large_fever_scifact'), paraphr

In [ ]:
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional

In [ ]:
from T5ParEvo.src.data import data as dataobj

def get_datalist_from_dict_ds(data_dict : Dict):
    counter_gold_claim = 0
    data_list = []
    for cur_claim in data_dict:
        claim = dataobj.Claim(claim_text = cur_claim['claim'])
        rationales = []
        for cur_ratnl in cur_claim['list_rationales']:
            rationale = dataobj.Rationale(rationale_text = cur_ratnl)
            label_obj = dataobj.Label.get_enum_rep_label(res_str = cur_claim['label'])
            rationales.append({'rationale': rationale, 'label': label_obj})
        claim_org = dataobj.ClaimRationale(id = str(counter_gold_claim), claim = claim, rationales = rationales)
        data_list.append(claim_org)    
        counter_gold_claim += 1    
    return data_list

In [ ]:
data_list_train = get_datalist_from_dict_ds(dic_train)



In [ ]:
data_list_train